In [10]:
!pip install torchsummary

In [2]:
import torch
import torch.nn as nn
from torchvision.transforms import transforms
from torchvision.datasets import ImageFolder
import torch.optim as optim




In [3]:

# data transformation
data_transform = transforms.Compose([
    transforms.Resize((80, 80)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


In [4]:

import os
dataset = torch.utils.data.DataLoader(
    ImageFolder(root='./Dataset/data', transform=data_transform),
    batch_size=32, shuffle=True, num_workers=4)

path_classes_csv = './image_dataset.csv'

import pandas as pd
df = pd.read_csv(path_classes_csv)




In [5]:

class SatelliteClassifier(nn.Module):
    def __init__(self, channels=3, num_classes=4):
        super(SatelliteClassifier, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(channels, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Linear(128 * 10 * 10, 512), 
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(512, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), 128 * 10 * 10)
        x = self.classifier(x)
        return x

    def save(self, path):
        torch.save(self.state_dict(), path)



In [6]:

model = SatelliteClassifier(channels=3)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# training loop
num_epochs = 10
for epoch in range(num_epochs):
    for inputs, labels in dataset:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


Epoch [1/10], Loss: 0.2992
Epoch [2/10], Loss: 0.4046
Epoch [3/10], Loss: 0.3714
Epoch [4/10], Loss: 0.4819
Epoch [5/10], Loss: 0.1290
Epoch [6/10], Loss: 0.1168
Epoch [7/10], Loss: 0.1087
Epoch [8/10], Loss: 0.0391
Epoch [9/10], Loss: 0.1470
Epoch [10/10], Loss: 0.1421


In [9]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in dataset:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f'Validation Accuracy: {accuracy * 100:.2f}%')


Validation Accuracy: 86.70%


In [12]:
model.save('./savemod.model')

In [13]:
import torch
from torchvision import transforms
from PIL import Image

image_path = "./cloud.jpg"

transform = transforms.Compose([
    transforms.Resize((80, 80)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

image = Image.open(image_path)
input_tensor = transform(image)
input_batch = input_tensor.unsqueeze(0)  

model = SatelliteClassifier(channels=3)
model.eval()

with torch.no_grad():
    output = model(input_batch)

_, predicted_class = torch.max(output, 1)
print("Predicted Class:", predicted_class.item())




Predicted Class: 3


In [ ]:

file_paths = [fname for fname in df["image_path"].tolist()]
file_paths

['./Dataset/data/cloudy\\train_10021.jpg',
 './Dataset/data/cloudy\\train_10043.jpg',
 './Dataset/data/cloudy\\train_10070.jpg',
 './Dataset/data/cloudy\\train_10081.jpg',
 './Dataset/data/cloudy\\train_10096.jpg',
 './Dataset/data/cloudy\\train_101.jpg',
 './Dataset/data/cloudy\\train_10128.jpg',
 './Dataset/data/cloudy\\train_10166.jpg',
 './Dataset/data/cloudy\\train_10173.jpg',
 './Dataset/data/cloudy\\train_10177.jpg',
 './Dataset/data/cloudy\\train_10201.jpg',
 './Dataset/data/cloudy\\train_10224.jpg',
 './Dataset/data/cloudy\\train_10225.jpg',
 './Dataset/data/cloudy\\train_10231.jpg',
 './Dataset/data/cloudy\\train_10263.jpg',
 './Dataset/data/cloudy\\train_10265.jpg',
 './Dataset/data/cloudy\\train_10269.jpg',
 './Dataset/data/cloudy\\train_10293.jpg',
 './Dataset/data/cloudy\\train_10326.jpg',
 './Dataset/data/cloudy\\train_10333.jpg',
 './Dataset/data/cloudy\\train_10389.jpg',
 './Dataset/data/cloudy\\train_10406.jpg',
 './Dataset/data/cloudy\\train_10421.jpg',
 './Dataset/d

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from sklearn.preprocessing import LabelEncoder

# Custom dataset class
class CustomDataset(Dataset):
    def __init__(self, file_paths, labels, transform=None):
        self.file_paths = file_paths
        self.labels = labels
        self.transform = transform
        self.label_encoder = LabelEncoder()
        self.labels_encoded = self.label_encoder.fit_transform(self.labels)

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        img_path = self.file_paths[idx]
        image = Image.open(img_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        label = torch.tensor(self.labels_encoded[idx], dtype=torch.long)
        return image, label

# Define data transformations
data_transform = transforms.Compose([
    transforms.Resize((80, 80)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


file_paths = [fname for fname in df["image_path"].tolist()]
labels = [label for label in df["label"].tolist()]

# Create a custom dataset
custom_dataset = CustomDataset(file_paths, labels, transform=data_transform)

# Create a DataLoader
batch_size = 4
data_loader = DataLoader(custom_dataset, batch_size=batch_size, shuffle=True)

# Define the CNN model
class WeatherClassifier(nn.Module):
    def __init__(self, num_classes=4):
        super(WeatherClassifier, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.batchnorm1 = nn.BatchNorm2d(32)
        self.relu1 = nn.ReLU(inplace=True)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.batchnorm2 = nn.BatchNorm2d(64)
        self.relu2 = nn.ReLU(inplace=True)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(64 * 20 * 20, 128)
        self.relu3 = nn.ReLU(inplace=True)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.pool1(self.relu1(self.batchnorm1(self.conv1(x))))
        x = self.pool2(self.relu2(self.batchnorm2(self.conv2(x))))
        x = x.view(-1, 64 * 20 * 20)
        x = self.relu3(self.fc1(x))
        x = self.fc2(x)
        return x

# Instantiate the model
model = WeatherClassifier(num_classes=4)

# Specify loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    for inputs, labels in data_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')


import torch
from torchvision import transforms
from PIL import Image

# Define the path to the image you want to predict
image_path = "./cloud.jpg"

# Load and preprocess the image
transform = transforms.Compose([
    transforms.Resize((80, 80)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])


image = Image.open(image_path)
input_tensor = transform(image)
input_batch = input_tensor.unsqueeze(0)  # Add batch dimension

# Load the trained model
model = WeatherClassifier(num_classes=4)
model.eval()

# Perform prediction
with torch.no_grad():
    output = model(input_batch)

# Get the predicted class
_, predicted_class = torch.max(output, 1)

# Print the predicted class
print("Predicted Class:", predicted_class.item())


Epoch [1/10], Loss: 0.6968
Epoch [2/10], Loss: 0.0995
Epoch [3/10], Loss: 0.2463
Epoch [4/10], Loss: 0.1363
Epoch [5/10], Loss: 0.5174
Epoch [6/10], Loss: 0.0344
Epoch [7/10], Loss: 0.0353
Epoch [8/10], Loss: 0.0034
Epoch [9/10], Loss: 0.0817
Epoch [10/10], Loss: 0.3521
Predicted Class: 2


In [ ]:
classes = [c for c in df["label"].unique()]
classes

['Cloudy', 'Desert', 'Green_Area', 'Water']